This notebook is devoted to developing a good algorithm for managing a complex mesh like ours. The main idea is to have a representation of a node that is a geometric object with its x y and z, as well as it's id in the global matrix

In [1]:
import numpy as np
from scipy.sparse.linalg import spsolve
from scipy.sparse import coo_matrix
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

from pyfe3d.shellprop_utils import isotropic_plate
from pyfe3d import Quad4, Quad4Data, Quad4Probe, INT, DOUBLE, DOF

In [ ]:
class MeshInfo():
    def __init__(self):
        self._nodeCount = 0
    
    def new_id(self): 
        '''gets an ID starting at 1. 
        This ID has to be assigned when created. 
        Not using it causes an error!'''
        self._nodeCount += 1
        return self._nodeCount

    @property
    def nodeCount(self):
        '''Use this to get the node count 
        without creating a new ID'''
        return self._nodeCount


class Node():
    def __init__(self, x:float, y:float, z:float, meshinfo:MeshInfo):
        self.id = meshinfo.new_id()
        self.x, self.y, self.z = x, y, z

Now, we will create a shrinking sheet and same one sheet riveted along y=0 of the normal coordinate system. We will try to encapsulate it as a class. first of all: the root

In [ ]:
import typing as ty
class Root():
    def __init__(self, nodes:ty.List[ty.Tuple[float]], exports:ty.List[function]):
        #nodes are the generated node locations, use gcl or something alike
        #exports are check function as function of x, y, z numpy arrays, that give checks for groups of export points
        self.meshinfo = MeshInfo() #one monkey said: new root, new mesh
        self.nodes = np.array([Node(node[0], node[1], node[2], self.meshinfo) for node in nodes])
        x = np.array([node.x for node in self.nodes])
        y = np.array([node.y for node in self.nodes])
        z = np.array([node.z for node in self.nodes])

        #creating export nodes
        self.exports = list()
        for fun in exports:
            check = fun(x,y,z)
            self.exports.append(self.nodes[check])
        